# ws 01

In [ ]:
import numpy as np
import cv2

cv2.__version__

In [ ]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg") #  
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# print(classes)    
layer_names = net.getLayerNames()

output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
len(classes)

In [ ]:
classes[:10]
classes

In [ ]:
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [ ]:
img = cv2.imread("city2.jpg")

img = cv2.resize(img, None, fx=0.9, fy=0.9)
height, width, channels = img.shape
img.shape

In [ ]:
import time
start = time.time()

img_size = (320, 320) 

blob = cv2.dnn.blobFromImage(img, 1/255, img_size, (0, 0, 0), True, crop=False)
net.setInput(blob)
outputs = net.forward(output_layers)

end = time.time()

print("Time Taken: {:.2f} secs".format(end - start))

In [ ]:
class_ids = []
confidences = []
boxes = []
for out in outputs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5: # 
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            x = int(center_x - w / 2)
            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
            
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX # font = cv2.FONT_HERSHEY_PLAIN
text_color = (255,255,255)

for i in range(len(boxes)):
    if i in indices:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        conf =  confidences[i] 
        label = '{} {:.1f}%'.format(label, conf*100)
        
        color = colors[i]

        if class_ids[i] == 7: 
            color=(115,48,30)
        elif class_ids[i]==1:
            color=(255,0,55)
        elif class_ids[i]==16:
            color=(2,245,55)            
            
        t_size = cv2.getTextSize(label, font, 1 , 1)[0] 
       
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2) 

           
        c2 = x + int(t_size[0]*0.8), y - int(t_size[1]*1.2) 
        
        cv2.rectangle(img, (x,y-2), c2, color, -1) 
        cv2.putText(img, label, (x, y-2), font, 0.7 , text_color, 3) 

cv2.imshow("Mywindow", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ws 02 

In [ ]:
import cv2
import numpy as np

colorGreen = (0,255,0)
img_size = (320, 320)
confThreshold = 0.5
nmsThreshold = 0.4
scale = 0.75  

# cap = cv2.VideoCapture(0) # 
# cap.set(3, 920) #
# cap.set(4, 520) #

cap = cv2.VideoCapture('vdo.mp4')  # 

if (not(cap.isOpened())):  
    print("Could not open the Cam/VDO file") 

classesFile = 'coco.names'
classNames = []

    
with open("coco.names", "r") as f:
    classNames = [line.strip() for line in f.readlines()]
    

modelConfiguration = 'yolov3.cfg'
modelWeights = 'yolov3.weights'


net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)


layerNames = net.getLayerNames()

outputNames = [layerNames[i[0]-1] for i in net.getUnconnectedOutLayers()]

def locateObjects(outputs,img):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []

    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                w,h = int(det[2]* wT), int(det[3]*hT)
                x,y = int((det[0]*wT)-w/2), int((det[1]*hT)-h/2)
                bbox.append([x,y,w,h])
                classIds.append(classId)
                confs.append(float(confidence))

    indices = cv2.dnn.NMSBoxes(bbox, confs, confThreshold, nmsThreshold)

    for i in indices:
        i = i[0]
        box = bbox[i]
        x,y,w,h = box[0], box[1], box[2], box[3]
        cv2.rectangle(img, (x,y),(x+w,y+h),colorGreen,2)
        label = str(classNames[classIds[i]])
        label = '{} {:.1f}%'.format(label, confs[i]*100)
        cv2.putText(img, label,
                    (x,y-8), cv2.FONT_HERSHEY_SIMPLEX, 1.1,colorGreen,2)

       
# Main Loop        
while (cap.isOpened()): # True:
    success, img = cap.read()  # img = frame

    
    blob = cv2.dnn.blobFromImage(img, 0.00392, img_size, (0,0,0), crop=False) 
    net.setInput(blob)

    outputs = net.forward(outputNames)

    locateObjects(outputs, img)

    cv2.imshow('Mywindow', img)
    
    #cv2.waitKey(0)
#     if cv2.waitKey(1) & 0xFF == 27:  # ESC to exit
    if cv2.waitKey(1) == 27: 
        break

cap.release()  
cv2.destroyAllWindows()   # Closes all the frames    